In [21]:
## 9, 99, 999 애들 null 처리하고 데이터 얼마나 빠지는지 체크
import pandas as pd

df = pd.read_csv('../data/HN__4_8__19__selected.csv')

/var/folders/3y/hj83hdwn2nn49jp7jks64t480000gn/T/ipykernel_34074/1267087970.py:4: DtypeWarning: Columns (205) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/HN__4_8__19__selected.csv')


In [22]:
# y라벨 결측치
# mh_PHQ_S    9074
# DF2_pr      1629
# BP_PHQ_1    1629
# BP_PHQ_2    1629
# BP_PHQ_3    1629
# BP_PHQ_4    1629
# BP_PHQ_5    1629
# BP_PHQ_6    1629
# BP_PHQ_7    1629
# BP_PHQ_8    1629
# BP_PHQ_9    1629
# dtype: int64

#TODO 결측 채워도 되는지 나중에 확인할것
# 총점: 0~27점

cond = df["BP_PHQ_1"].notna() & \
        df["BP_PHQ_2"].notna() & \
        df["BP_PHQ_3"].notna() & \
        df["BP_PHQ_4"].notna() & \
        df["BP_PHQ_5"].notna() & \
        df["BP_PHQ_6"].notna() & \
        df["BP_PHQ_7"].notna() & \
        df["BP_PHQ_8"].notna() & \
        df["BP_PHQ_9"].notna()

df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]].value_counts()

BP_PHQ_1  BP_PHQ_2  BP_PHQ_3  BP_PHQ_4  BP_PHQ_5  BP_PHQ_6  BP_PHQ_7  BP_PHQ_8  BP_PHQ_9
9.0       9.0       9.0       9.0       9.0       9.0       9.0       9.0       9.0         1567
0.0       0.0       0.0       9.0       0.0       0.0       0.0       0.0       0.0            9
          9.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0            5
          0.0       0.0       0.0       0.0       9.0       0.0       0.0       0.0            4
1.0       9.0       9.0       9.0       9.0       9.0       9.0       9.0       9.0            3
                                                                                            ... 
0.0       1.0       2.0       1.0       0.0       1.0       9.0       9.0       1.0            1
                    1.0       9.0       0.0       0.0       0.0       0.0       1.0            1
                              2.0       9.0       2.0       0.0       0.0       0.0            1
                    0.0       0.0     

In [23]:
a = df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]]

b = a[a <= 3].sum(axis=1)
b[b >= 1] #125

df['mh_PHQ_S'] = b[b >=1]

In [24]:
df.select_dtypes(include=['object']).columns


####### 얘네 빼도 되는지 확인 - object 타입인 녀석들 ####
df[['LQ4_24', 'N_DT_DS', 'N_DUSUAL', 'N_DAY']]


,LQ4_24,N_DT_DS,N_DUSUAL,N_DAY
0,NaN,NaN,3.0,수요일
1,NaN,NaN,2.0,수요일
2,NaN,NaN,1.0,화요일
3,NaN,NaN,1.0,화요일
4,NaN,NaN,NaN,NaN
...,...,...,...,...
94114,NaN,NaN,2.0,È­
94115,NaN,NaN,3.0,È­
94116,NaN,NaN,2.0,¼ö
94117,NaN,NaN,3.0,¼ö


In [14]:
####### 9, 99, 999 인 녀석들 null 처리
import numpy as np
df = df.replace([9, 99, 999, 9999], np.nan, inplace=False)


In [25]:
# row 기준 
threshold = 0.3 # threshold for null proportion
null_percent = df.isnull().mean(axis = 1) # 각 행에 대한 결측값 비율 계산
df = df[null_percent < threshold]

# Define depression (dependent variable)
cond1 = df['mh_PHQ_S'] >= 10 # total score 
cond2 = df['BP_PHQ_9'].isin([1, 2, 3]) # person who chose 1, 2, 3 in 9th question

df[cond1 | cond2].drop_duplicates() # person with cond1 or cond2


# Create a new variable 'depressed' 
df.loc[cond1 | cond2, 'depressed'] = 1 # 두 조건 중 하나 이상을 만족하는 행에 'depressed' 변수에 1 값 
df.loc[~(cond1 | cond2), 'depressed'] = 0 # 두 조건 중 하나라도 만족하지 않는 행에 0 값


# Remove people who might be depressed
# DF2_pr == 8 (never been diagnosed by doctor) & mh_PHQ_S == NaN) 인 경우를 제외하려면?
# DF2_pr != 8 or mh_PHQ != NaN (전부 다 ~ 취하기)
df = df[(df['DF2_pr'] != 8 |  ~df['mh_PHQ_S'].isna())]
df.shape


# Remove columns if percentage of NaN exceeds 0.1
threshold = 0.3 # 이거로 하나, 0.1로 하나 큰 차이 x
threshold = 0.1 

null_percent = df.isnull().mean() # 각 열에 대한 결측값 비율 계산
selected_cols = null_percent[null_percent < threshold].index
df = df[selected_cols]
df.shape # (20492, 314)


df.groupby('depressed').size()


depressed
0.0    82952
1.0     1165
dtype: int64

In [ ]:
#TODO fillna
#TODO scaling
#TODO over, under sampling

In [ ]:
from sklearn.impute import KNNImputer


mh_PHQ_S_col = df8["mh_PHQ_S"]


imputer = KNNImputer(n_neighbors=20)  
mh_PHQ_S_col_filled = imputer.fit_transform(mh_PHQ_S_col.values.reshape(-1, 1))


df8["mh_PHQ_S"] = mh_PHQ_S_col_filled

In [ ]:

# df2 = df.dropna()
# "depressed" in df2.columns # True

In [19]:
df2.shape

(39617, 157)

In [20]:
df2.groupby('depressed').size()

depressed
0.0    39004
1.0      613
dtype: int64

In [16]:
#TODO 오버샘플링 or 언더

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import pandas as pd

# X, y = make_classification(n_classes=2, class_sep=2,
#                            weights=[0.1, 0.9], n_informative=3, n_redundant=1, 
#                            flip_y=0, n_features=20, n_clusters_per_class=1, 
#                            n_samples=1000, random_state=10)


######
X = df2.drop('depressed', axis = 1)
y = df2['depressed']

# 원본 데이터셋의 클래스 분포 확인
print('Original dataset shape %s' % Counter(y))

# SMOTE 적용
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# 오버샘플링 후의 클래스 분포 확인
print('Resampled dataset shape %s' % Counter(y_res))

# TODO 데이터 시각화
temp = pd.DataFrame(X_res)
temp['y_label'] = y_res

# plt.scatter(temp[temp['label']==0], temp[temp['label']==0], label='0')
# plt.scatter(temp[temp['label']==1], temp[temp['label']==1], label='1')
# plt.title("SMOTE - Over-sampled Data")
# plt.legend()
# plt.show()

Original dataset shape Counter({0.0: 39004, 1.0: 613})


/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be 

Resampled dataset shape Counter({0.0: 39004, 1.0: 39004})


/var/folders/3y/hj83hdwn2nn49jp7jks64t480000gn/T/ipykernel_34074/2805469819.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp['y_label'] = y_res


In [17]:
temp.shape

(78008, 157)

In [ ]:
# temp.drop('y_label', axis=1).to_csv('../data/HN_train_X.csv', index=False)
# temp["y_label"].to_csv('../data/HN_train_y.csv', index=False)